In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/COVID-19_Radiography_Dataset"

### Important Libraries 

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

### Image Preprocessing using Canny Edge Detector

In [ ]:
def pre_processing(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.Canny(image, 80, 100)
    return image

In [ ]:
tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    preprocessing_function= pre_processing
)

### Image Data Generators 

In [ ]:
train=path+"/Train"
val=path+"/Val"
test=path+"/Test"

In [ ]:
gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (299, 299))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(299, 299))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(299, 299))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


### Transfer Learning with VGG19 Base Model



In [ ]:
base_model=tf.keras.applications.InceptionResNetV2(
    weights="imagenet",
    input_shape=(299,299,3),
    classifier_activation="softmax",
)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512,activation= 'relu'))
model.add(Dropout(0.15))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.15)) 
model.add(Dense(2, activation='softmax'))
model.summary()

225209952/225209952 [==============================] - 11s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1000)             55873736  
 onal)                                                           
                                                                 
 flatten_1 (Flatten)         (None, 1000)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               512512    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               65664     
                                                                 
 dropout_3 (Dropout)         (None, 128)            

In [ ]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)
history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 50, epochs = 12, callbacks = callback)

Epoch 1/12
50/50 [==============================] - 1331s 26s/step - loss: 0.6138 - accuracy: 0.7138 - val_loss: 0.4043 - val_accuracy: 0.8740
Epoch 2/12
50/50 [==============================] - 457s 9s/step - loss: 0.4995 - accuracy: 0.7541 - val_loss: 0.3398 - val_accuracy: 0.8837
Epoch 3/12
50/50 [==============================] - 252s 5s/step - loss: 0.4620 - accuracy: 0.7764 - val_loss: 0.3670 - val_accuracy: 0.8352
Epoch 4/12
50/50 [==============================] - 148s 3s/step - loss: 0.4409 - accuracy: 0.7925 - val_loss: 0.3672 - val_accuracy: 0.8283
Epoch 5/12
50/50 [==============================] - 105s 2s/step - loss: 0.4283 - accuracy: 0.7966 - val_loss: 0.3589 - val_accuracy: 0.8158
Epoch 6/12
50/50 [==============================] - 93s 2s/step - loss: 0.4181 - accuracy: 0.8058 - val_loss: 0.3848 - val_accuracy: 0.7784
Epoch 7/12
50/50 [==============================] - 85s 2s/step - loss: 0.3930 - accuracy: 0.8232 - val_loss: 0.3754 - val_accuracy: 0.7909
Epoch 8/12
50

In [ ]:
model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 50, epochs = 1, callbacks = callback)

50/50 [==============================] - 79s 2s/step - loss: 0.3772 - accuracy: 0.8313 - val_loss: 0.3972 - val_accuracy: 0.7867


### Save the Model

In [ ]:
model.save_weights("InceptionResNet.h5")

In [ ]:
y_pred = model.predict(test_gen)

12/12 [==============================] - 7s 598ms/step


In [ ]:
y_pred_list = []

for i in range(len(y_pred)):
    y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_pred_list.append(y_pred[i][0])


<ipython-input-25-aaf20d1ce428>:4: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))


In [ ]:
y_actual = []

for i in range(len(y_pred)):
  if i <361 :
    y_actual.append(0)
  else:
    y_actual.append(1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_actual,y_pred_list)

0.7368421052631579